In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath(".."))
sys.path.append(os.path.abspath("../backend"))
sys.path.append(os.path.abspath(""))

In [3]:

from rdflib.plugins.stores.sparqlstore import SPARQLStore
from backend.model import *
from backend.ontology import *
from backend.datasetmatcher import *
from backend.explorative_support import *

base_path = "../data"
onto_path = f"{base_path}/hero-ontology/hereditary_clinical.ttl"


# graph = Graph().parse(onto_path, format="turtle")
# graph.bind("bto", "http://www.semanticweb.org/ontologies/2020/3/bto#")

# store = SPARQLStore(
#     "http://localhost:7200/repositories/dpedia",
#     method="POST_FORM",
#     params={"infer": False, "sameAs": False},
# )

# store = SPARQLStore(
#     "http://localhost:3030/dbpedia/query",
#     method="POST_FORM",
#     params={"infer": False, "sameAs": False},
# )
store = SPARQLStore(
    "http://localhost:7012/",
    method="POST_FORM",
    params={"infer": False, "sameAs": False},
)
graph = Graph(store=store)

config = OntologyConfig()

ontology_manager = OntologyManager(config, graph)
dataset_manager = DatasetManager(ontology_manager)
dataset_manager.initialise(glob_path="data/datasets/ALS/**/*.csv")

# ontology_manager.load_full_graph()


-- Loading and merging datasets
0 datasets loaded


In [5]:
topic_man = TopicModelling(ontology_manager)
print(topic_man.identifier)
topic_man.initialize_topics(force=True, delete_tables=True)

bb3ed8536753defab35ef3a106ad6561d70669359971c082b1b743c485365110
Initializing topics


Embedding classes:   0%|          | 0/790 [00:00<?, ?it/s]Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Embedding relations: 100%|██████████| 790/790 [03:01<00:00,  4.35it/s]


ConnectionError: (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: d71a20eb-1f48-46dd-a39a-8f404b93fffe)')

In [7]:
topic_man = TopicModelling(ontology_manager)
topic_man.model_topics()

llama_load_model_from_file: using device Metal (Apple M2 Pro) - 7643 MiB free
llama_model_loader: loaded meta data with 27 key-value pairs and 292 tensors from /Users/benedikt/.cache/huggingface/hub/models--NousResearch--Hermes-3-Llama-3.1-8B-GGUF/snapshots/307a5dfb59aa38d88b6cfd32f44b8ad7c1da9fb8/./Hermes-3-Llama-3.1-8B.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Hermes 3 Llama 3.1 8B
llama_model_loader: - kv   3:                       general.organization str              = NousResearch
llama_model_loader: - kv   4:                           general.basename str              = Hermes-3-Llama-3.1
llama_model_load

Batches:   0%|          | 0/80 [00:00<?, ?it/s]

2024-12-20 09:11:26,732 - BERTopic - Embedding - Completed ✓
2024-12-20 09:11:26,732 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2024-12-20 09:11:34,764 - BERTopic - Dimensionality - Completed ✓
2024-12-20 09:11:34,764 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-12-20 09:11:34,812 - BERTopic - Cluster - Completed ✓
2024-12-20 09:11:34,817 - BERTopic - Representation - Extracting topics from clusters using representation models.
  0%|          | 0/100 [00:00<?, ?it/s]llama_perf_context_print:        load time =    8112.02 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   442 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     5 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    8363.35 ms /   4

In [ ]:
with Session(topic_man.engine) as session:
    root_topics = session.execute(
        select(TopicDB).where(
            TopicDB.parent_topic_id == None,
            TopicDB.onto_hash == topic_man.identifier,
            len(TopicDB.sub_topics) > 0,
        )
    ).fetchall()
    print([(topic[0].topic, topic[0].sub_topics) for topic in root_topics], len(root_topics))

In [ ]:
docs = topic_man.build_docs()
# Create an instance of the Llama class and load the model

# Create the provider by passing the Llama class instance to the LlamaCppPythonProvider class
representation_llama = LlamaCPP(
    topic_man.llama_model, TOPIC_LLAMA3_PROMPT, diversity=0.3
)
topic_model_llm = BERTopic(
    embedding_model=topic_man.embedding_model,
    verbose=True,
    representation_model=representation_llama,
)
topic_model_llm.fit(docs["doc"])
hierarchical_topics = topic_model_llm.hierarchical_topics(docs["doc"])
topics = topic_model_llm.get_topic_info()


In [ ]:
hierarchical_topics

In [ ]:
topics[topics["Topic"].isin([6,9])]

In [ ]:
topic_model_llm.visualize_hierarchy(hierarchical_topics=hierarchical_topics)


In [ ]:

doc_info = topic_model_llm.get_document_info(docs["doc"], docs)
doc_info


In [ ]:
session = Session(topic_man.engine)
for i, doc in doc_info.iterrows():
    if doc["type"] == "subclass":
        subject = session.execute(
            select(SubjectInDB).where(SubjectInDB.subject_id == doc["subject_id"])
        ).first()
        topic_id = topics[topics["Topic"] == doc["Topic"]].index[0]
        if subject is not None:
            print(subject[0].subject_id, topic_id)
            subject[0].topic_id = topic_id

In [ ]:
doc_info

In [ ]:
# Save the model
# topic_model_llm.save(f"model_{topic_man.identifier}.pkl")
with Session(topic_man.engine) as session:
    # https://stackoverflow.com/a/48057795
    # defer for *current* transaction!
    session.execute(text("SET CONSTRAINTS ALL DEFERRED"))
    session.execute(
        delete(TopicDB).where(TopicDB.onto_hash == topic_man.identifier)
    )
    topic_map: dict[int, TopicDB] = {}
    for i, topic in topics.iterrows():
        topic_id=int(topic["Topic"])
        topic_label = topic_model_llm.get_topic(topic_id)[0][0]
        repr_docs = "\n".join(topic["Representative_Docs"])

        topic_embedding = topic_man.embedding_model.encode(
            f"{topic_label}\n\n{repr_docs}"
        )
        topic_db = TopicDB(
            topic_id=topic_id,
            topic=topic_label,
            doc_string=repr_docs,
            embedding=topic_embedding,
        )
            # elif doc["type"] == "property":
            #     link=session.execute(
            #         select(SubjectLinkDB).where(SubjectLinkDB.property_id == doc["property_id"])
            #     ).first()
            #     if link is not None:
            #         topic_db.subjects.append(link)
        topic_map[topic_id] = topic_db
        
    doc_info = topic_model_llm.get_document_info(docs["doc"], docs)
    for _, doc in doc_info.iterrows():
        if doc["type"] == "subclass":
            subject = session.execute(
                select(SubjectInDB).where(
                    SubjectInDB.subject_id == doc["subject_id"]
                )
            ).first()
            if subject is not None and doc["Topic"] in topic_map:
                subject[0].topic_id = doc["Topic"]
    for i, topic in hierarchical_topics.iterrows():
        parent_name: str = topic["Parent_Name"]
        parent_name = parent_name.strip("_")
        id = int(topic["Parent_ID"])
        topic_db = TopicDB(topic_id=id, topic=parent_name)
        topic_map[id] = topic_db
    for i, topic in hierarchical_topics.iterrows():
        sub_topics = [
            topic_map[int(topic[st])]
            for st in ["Child_Left_ID", "Child_Right_ID"]
        ]
        for sub_topic in sub_topics:
            topic_map[int(sub_topic.topic_id)].parent_topic_id = int(
                topic["Parent_ID"]
            )

    for topic in topic_map.values():
        topic.onto_hash = topic_man.identifier
        # if topic.embedding is None:
        #     topic.embedding = topic_man.embedding_model.encode(topic.topic)
        topic.onto_hash = topic_man.identifier
        session.add(topic)
    session.commit()

    # Aggregate embeddings
    # --> seems to be a bit too average-y -> parent topics are now combined
    root_topic = session.execute(
        select(TopicDB).where(
            TopicDB.onto_hash == topic_man.identifier,
            TopicDB.parent_topic_id == None,
        )
    ).first()

    def get_and_set_aggregated_embedding(topic: TopicDB) -> list[str]:
        if len(topic.sub_topics) == 0:
            return [topic.topic + "\n" + topic.doc_string]
        sub_topics = [
            topic_str
            for sub_topic in topic.sub_topics
            for topic_str in get_and_set_aggregated_embedding(sub_topic)
        ]
        topic_description = "\n".join(sub_topics)
        aggregated_embedding = topic_man.embedding_model.encode(
            f"{topic.topic}\n\n{topic_description}"
        )
        topic.embedding = aggregated_embedding
        return [topic.topic] + sub_topics

    get_and_set_aggregated_embedding(root_topic[0])
    session.commit()